# RNN
- 시계열 데이터를 처리하기에 좋은 뉴럴 네트워크 구조 (Recarrent Neural Network)
- Example
    - 번역기
    - 음악 생성기
    - 음성 인식
    - DNA 염기서열 분석

## RNN 특징
- CNN이 이미지 구역별로 같은 weight를 공유한다면,
- RNN은 시간 별로 같은 weight을 공유한다.
- 즉, 과거와 현재는 같은 weight를 공유한다.

## First Order System
- 현재 시간의 상태가 이전 시간의 상태와 관련이 있다고 가정
    - First-Order System
        $ x_t = f(x_(t-1)) $
    - 그림으로 표현하면
    - 이 시스템은 `**외부 입력**` 없이 자기 혼자서 돌아간다.
    - autonomous system
- 현재 시간의 상태가 이전 시간의 상태와, 현재의 입력에 관계가 있는 경우
    - 상태: x, 입력: u
    - 식으로 표현하면
        $ x_t = f(x_(t-1), u_t) $


## State-Space Model
- 어떤 시스템을 해석하기 위한 3요소:
    - $ x_t = f(x_(t-1), u_t) $
    - $ y_t = h(x_t) $









In [1]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
    return e_x / np.sum(e_x, axis=0, keepdims=True)

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
   Wax = parameters["Wax"]
   Waa = parameters["Waa"]
   Wya = parameters["Wya"]
   ba = parameters["ba"]
   by = parameters["by"]

   a_next = np.tanh(Wax @ xt + Waa @ a_prev + ba)
   yt_pred = softmax(Wya @ a_next + by)

   cache = (a_next, a_prev, xt, parameters)
   return a_next, yt_pred, cache

In [3]:
def rnn_forward(x, a0, parameters):

    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape

    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    a_next = a0
    for t in range(T_x):
        xt = x[:, :, t]
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, parameters)
        a[:, :, t] = a_next
        y_pred[:, :, t] = yt_pred
        caches.append(cache)

    caches = (caches, x)
    return a, y_pred, caches

In [4]:
# 테스트 코드
np.random.seed(1)
n_x, m, T_x = 3, 10, 4 # feature: 3개, batch size 10개, 시퀸스 4개
n_a, n_y = 5, 2 # Hidden state vector = 5, 최종 예측값의 종류 2
x_test = np.random.randn(n_x, m, T_x)
a0_test = np.random.randn(n_a, m)
parameters_test = {
    "Waa": np.random.randn(n_a, n_a),
    "Wax": np.random.randn(n_a, n_x),
    "Wya": np.random.randn(n_y, n_a),
    "ba": np.random.randn(n_a, 1),
    "by": np.random.randn(n_y, 1)
}
a_out, y_pred_out, caches_out = rnn_forward(x_test, a0_test, parameters_test)
print("a_out[4] = \n", a_out[4])
print("y_pred_out[1] = \n", y_pred_out[1])

a_out[4] = 
 [[-0.99935897 -0.57882882  0.99953622  0.99692362]
 [-0.99999375  0.77911235 -0.99861469 -0.99833267]
 [ 0.98895163  0.9905525   0.87805502  0.99623046]
 [ 0.9999802   0.99693738  0.99745184  0.97406138]
 [-0.9912801   0.98087418  0.76076959  0.54482277]
 [ 0.74865774 -0.59005528 -0.97721203  0.92063859]
 [-0.96279238 -0.99825059  0.95668547 -0.76146336]
 [-0.99251598 -0.95934467 -0.97402324  0.99861032]
 [ 0.93272501  0.81262652  0.65510908  0.69252916]
 [-0.1343305  -0.99995298 -0.9994704  -0.98612292]]
y_pred_out[1] = 
 [[8.70631878e-04 1.09227408e-01 2.95793685e-01 8.02699998e-02]
 [6.05834882e-04 6.73052187e-01 1.21038427e-03 1.17806974e-02]
 [5.72253732e-03 4.00062909e-03 2.06047094e-03 7.23375910e-01]
 [7.95603732e-01 8.62248606e-01 1.11182569e-01 8.15159466e-01]
 [2.57915964e-01 5.83969344e-01 9.40379273e-01 4.35479788e-02]
 [6.56311720e-01 1.13248528e-03 1.02336394e-02 1.00862411e-01]
 [6.07472236e-02 2.92414498e-01 1.14848948e-01 6.65231284e-01]
 [6.44953539e-02 